In [8]:
from roboflow import Roboflow
import os
from dotenv import load_dotenv
load_dotenv()

rf = Roboflow(api_key="M3Zhnyp687xQRSVg6vct")
project = rf.workspace("fyp-3bwmg").project("reloc2-den7l")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to reloc2-1 in yolov8:: 100%|██████████| 2948/2948 [00:00<00:00, 4135.04it/s]


In [10]:
from ultralytics import YOLO
model = YOLO("yolo11n-pose.pt")

augumentations = {#"hsv_h": 0.0,
                  #"hsv_s": 0.0,
                  #"hsv_v": 0.0,
                  "translate": 0.0,
                  "scale": 0.0,
                  #"shear": 0.0,
                  #"perspective": 0.0,
                  "fliplr": 0.0,
                  "flipud": 0.0,
                  "mosaic": 0.0,
                  "mixup": 0.0,
                  "cutmix": 0.0,
                  #"erasing": 0.0,
                  "copy_paste": 0.0}

model.train(data="reloc2-1/data.yaml",
            epochs=500,
            device=0,
            plots=True,
            # imgsz=640,
            **augumentations,
            batch=8,)

New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.224 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=reloc2-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-pose.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train7, nbs=64, nms=False,

ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7facab4d4b30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034, 

In [11]:
import numpy as np

full_court_x_m = 28
full_court_y_m = 15
delta_y_0_1_m = 0.9
box_y_m = 4.8
box_x_m = 5.79

full_court_x = 1
full_court_y = full_court_y_m/full_court_x_m

delta_y_0_1 = delta_y_0_1_m/full_court_y_m
box_y       = box_y_m/full_court_x_m
box_x       = box_x_m/full_court_x_m

reference_keypoints = np.array([[0, 0],
                                [0, delta_y_0_1],
                                [0, full_court_y/2 - box_y/2],
                                [0, full_court_y/2 + box_y/2],
                                [0, full_court_y - delta_y_0_1],
                                [0, full_court_y],
                                [full_court_x/2, full_court_y],
                                [full_court_x/2, 0],
                                [box_x, full_court_y/2 - box_y/2],
                                [box_x, full_court_y/2 + box_y/2],
                                [full_court_x, full_court_y],
                                [full_court_x, full_court_y - delta_y_0_1],
                                [full_court_x, full_court_y/2 + box_y/2],
                                [full_court_x, full_court_y/2 - box_y/2],
                                [full_court_x, delta_y_0_1],   
                                [full_court_x, 0],
                                [full_court_x - box_x, full_court_y/2 - box_y/2],
                                [full_court_x - box_x, full_court_y/2 + box_y/2]]) 

In [ ]:
from ultralytics import YOLO
from court_stitch import load_frames
import cv2

# --- Load YOLO model ---
# For keypoints (pose): use "yolo11n-pose.pt" or your trained weights
model = YOLO("runs/pose/train7/weights/best.pt")  # or "runs/pose/train/weights/best.pt"

# --- Source (can be image, folder, video, webcam index) ---
source = "input_videos/video_1.mp4"   # could also be "0" for webcam or "image.jpg"

frames = load_frames(source, sample_rate=1)
# --- Run Inference ---
for frame in frames:
    results = model.predict(
        source=frame,     # path to file, directory, or camera stream
        conf=0.5,          # confidence threshold
        # imgsz=640,
    )

    keypoints = results[0].keypoints

    if keypoints is None or len(keypoints.xy) == 0:
        print("No detections in this frame — skipping.")
        continue

    frame_keypoints = keypoints.xy.cpu().numpy()[0]
    conf = keypoints.conf.cpu().numpy()[0]

    conf_thresh = 0.3
    kp_scale = 600

    valid_ref_kp = reference_keypoints[conf > conf_thresh] * kp_scale
    valid_frame_kp = frame_keypoints[conf > conf_thresh]

    if len(valid_frame_kp) >= 4:
        tmp = frame.copy()
        for (x, y) in valid_frame_kp:
            cv2.circle(tmp, (int(x), int(y)), 4, (0,255,0), -1)

        cv2.imshow("Frame points", tmp)

        blank = np.ones((int(full_court_y * kp_scale), int(full_court_x * kp_scale), 3), np.uint8)*255
        for (x, y) in valid_ref_kp:
            cv2.circle(blank, (int(x), int(y)), 4, (0,0,255), -1)
        
        cv2.imshow("Ref points", blank) 

        H, _ = cv2.findHomography(valid_frame_kp, valid_ref_kp, cv2.RANSAC, 5.0)
        if H is not None:
            img2_warped = cv2.warpPerspective(frame, H, (int(full_court_x * 600), int(full_court_y * 600)))
            print(valid_frame_kp)
            cv2.imshow('Warped Image', img2_warped)
        key = cv2.waitKey(0)
    
cv2.destroyAllWindows()


[INFO] Loaded 117 frames.

0: 384x640 1 basketball, 5.4ms
Speed: 0.9ms preprocess, 5.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
[[     462.91      244.05]
 [     444.12      255.64]
 [     347.31      315.26]
 [     198.09      406.04]
 [     29.599      505.54]
 [          0      529.16]
 [     725.88      349.75]
 [     635.23      444.23]]

0: 384x640 1 basketball, 5.4ms
Speed: 1.1ms preprocess, 5.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[[     462.32      240.72]
 [     443.43      252.61]
 [      347.1      312.97]
 [     196.69      405.37]
 [     31.136      506.32]
 [          0      530.21]
 [     725.17      348.51]
 [     634.39      445.41]]

0: 384x640 1 basketball, 5.3ms
Speed: 1.1ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[[     462.14       237.3]
 [     442.94      249.72]
 [     345.31      311.55]
 [     192.25      406.79]
 [     26.988      510.91]
 [          0      53